## SquamataSB_UploadMultipleChildren - Jupyter notebook for batch child creation and uploading files to data to ScienceBase

This module performs the following operations:
- Create list of upload data directories.
- Get a listing of child catalog items.
- Match upload directory to child item. 
- Replace existing files on ScienceBase with all files in upload directory.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

## SquamataSB_AMT version history
v1.0 
- First created for MT data release: 

- This module makes extensive use of ScienceBase Python: 
  - https://pypi.org/project/sciencebasepy/
  - https://github.com/usgs/sciencebasepy


In [1]:
# Phil Brown (pbrown@usgs.gov) 2021 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [2]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [3]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

# Load required Libraries
from lxml import etree
import sciencebasepy as pysb
import glob
#Create Function that allows one to implement markdown display when printing
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
#Set Data Paths
mtDataPath = r"C:\CurrentWork\DataReleases\EU-Mike\child_xmls" #The 'r' signifies a string literal. Use for paths.
#Check Paths for the fun of it
print ('The MT Data Path is: ' + '"' + mtDataPath + '"')

The MT Data Path is: "C:\CurrentWork\DataReleases\EU-Mike\child_xmls"


In [6]:
#Login to ScienceBase
sb = pysb.SbSession()
username = input("Username:  ")
sb.loginc(str(username))
#Check to see if login is successful
sb.is_logged_in()
#Get the ScienceBase Login session info
sb.get_session_info()

Username:  pbrown@usgs.gov
········


{'fullDisplayName': 'Philip J Brown II [pbrown@usgs.gov]',
 'isLoggedIn': True,
 'jossoSessionId': 'FF353C6CDACCFAE3F36497B103BD69FA',
 'displayName': 'Philip J Brown II',
 'email': 'pbrown@usgs.gov',
 'username': 'pbrown@usgs.gov'}

In [7]:
#Open parent page json item.
catNum = '5e87586b82cee42d13400f6b' #!!!!!MAKE SURE THIS IS THE CORRECT SB CATALOG NUMBER!!!!!
item_json = sb.get_item(catNum)
print(json.dumps(item_json, indent=2, sort_keys=True))


{
  "body": "This dataset consists of 176 wideband magnetotelluric (MT) stations collected from 2015-2019 across parts of Minnesota, Wisconsin and the Upper Peninsula of Michigan. The U.S. Geological Survey acquired these data as part of regional investigations into the geologic and tectonic framework of the area and to support mineral resource investigations. These data have been used to generate a 3D regional conductivity model of the area. Files included in this publication include measured electric- and magnetic-field time series as well as estimated impedance and vertical-magnetic field transfer functions.",
  "browseCategories": [
    "Data Release - In Progress",
    "Data"
  ],
  "browseTypes": [
    "Shapefile",
    "Downloadable",
    "Map Service",
    "OGC WMS Layer",
    "OGC WFS Layer"
  ],
  "citation": "Paul A. Bedrosian, Michael D. M. Pace, Evan M. Cox, Katrina D. Zamudio, and Daniel W. Feucht, 20200814, Magnetotelluric data from Minnesota, Wisconsin, and Upper Michiga

In [8]:
#Get all the children catalog numbers
children = sb.get_child_ids(catNum)
#Print array
for i in range(len(children)): 
 print (children[i])

5f74a82082cef8d18399bf33
5f74a26f82cef8d18399b70b
5f6aa2f282ce38aaa244968a
5f74b72882cef8d18399bfe6
5f74c15682cef8d18399e6c8
5f74c68682cef8d18399e7e2
5f74c35f82cef8d18399e7d1
5f6ac7fa82ce38aaa2449742
5f6ad1bb82ce38aaa2449779
5f6ae07282ce38aaa2449b0d
5f74b1d282cef8d18399bfac
5f74bdc882cef8d18399d397
5f6acfad82ce38aaa2449770
5f6ae17282ce38aaa2449b14
5f73b9b182cef8d183996071
5f6acd3a82ce38aaa2449767
5f6aeaf382ce38aaa2449b4c
5f73c58c82cef8d1839960b3
5f6acea482ce38aaa244976c
5f6ae5a082ce38aaa2449b33
5f74af7182cef8d18399bf90
5f6a83a582ce38aaa24495e2
5f6a8c4a82ce38aaa244961a
5f6a6ef082ce38aaa244950f
5f6a93b282ce38aaa244963e
5f6a9d2b82ce38aaa2449666
5f6a692e82ce38aaa24494af
5f6a683582ce38aaa244949a
5f6a5dee82ce38aaa24493f5
5f6a5d2382ce38aaa24493ec
5f6a653b82ce38aaa2449471
5f6a5c1b82ce38aaa24493e7
5f74c47d82cef8d18399e7d7
5f748d1382cef8d183998f3d
5f74c59182cef8d18399e7dc
5f74ae8b82cef8d18399bf82
5f6a8e5382ce38aaa2449627
5f73c0d882cef8d1839960a3
5f6a6c5982ce38aaa24494f0
5f6a772182ce38aaa2449561


In [9]:
#Create an array of child items titles to compare the upload file names to.
arrSBTitle = []
for i in range(len(children)): 
 item_json = sb.get_item(children[i])
 arrSBTitle.append (item_json['title'].split(' ')[1])
 print ('*** ' + children[i] + ', ' + item_json['title'].split(' ')[1] + ' ***\n======================')
 print ('\n------------------------------------------------------------------------------------------------')


*** 5f74a82082cef8d18399bf33, pen010 ***

------------------------------------------------------------------------------------------------
*** 5f74a26f82cef8d18399b70b, pen007 ***

------------------------------------------------------------------------------------------------
*** 5f6aa2f282ce38aaa244968a, nmn926 ***

------------------------------------------------------------------------------------------------
*** 5f74b72882cef8d18399bfe6, pen107 ***

------------------------------------------------------------------------------------------------
*** 5f74c15682cef8d18399e6c8, pen116 ***

------------------------------------------------------------------------------------------------
*** 5f74c68682cef8d18399e7e2, pen120 ***

------------------------------------------------------------------------------------------------
*** 5f74c35f82cef8d18399e7d1, pen117 ***

------------------------------------------------------------------------------------------------
*** 5f6ac7fa82ce38aaa244974

*** 5f74b9cf82cef8d18399bff7, pen109 ***

------------------------------------------------------------------------------------------------
*** 5f74ad1382cef8d18399bf71, pen012 ***

------------------------------------------------------------------------------------------------
*** 5f6a809782ce38aaa24495cd, nmn708 ***

------------------------------------------------------------------------------------------------
*** 5f6a8d4d82ce38aaa2449620, nmn903 ***

------------------------------------------------------------------------------------------------
*** 5f6a8f6082ce38aaa244962e, nmn905 ***

------------------------------------------------------------------------------------------------
*** 5f6a8b1d82ce38aaa2449612, nmn901 ***

------------------------------------------------------------------------------------------------
*** 5f6a77ca82ce38aaa244956e, eup628 ***

------------------------------------------------------------------------------------------------
*** 5f73ce3782cef8d18399629

*** 5f73b11082cef8d183995ff1, wup101 ***

------------------------------------------------------------------------------------------------
*** 5f73d6d182cef8d1839962ae, wup220 ***

------------------------------------------------------------------------------------------------
*** 5f73b70282cef8d183996048, wup105 ***

------------------------------------------------------------------------------------------------
*** 5f6ad76782ce38aaa244980c, wup501 ***

------------------------------------------------------------------------------------------------
*** 5f6adadd82ce38aaa24499cf, wup502 ***

------------------------------------------------------------------------------------------------
*** 5f73c82a82cef8d1839960db, wup205 ***

------------------------------------------------------------------------------------------------
*** 5f6ae29582ce38aaa2449b19, wup510 ***

------------------------------------------------------------------------------------------------
*** 5f73c70182cef8d1839960b

*** 5f6a9f1e82ce38aaa2449677, nmn922 ***

------------------------------------------------------------------------------------------------
*** 5f6aa0d882ce38aaa2449682, nmn924 ***

------------------------------------------------------------------------------------------------
*** 5f6a9a2982ce38aaa244965a, nmn916 ***

------------------------------------------------------------------------------------------------
*** 5f6a7b8d82ce38aaa24495a0, nmn702 ***

------------------------------------------------------------------------------------------------
*** 5f6a881382ce38aaa2449600, nmn807 ***

------------------------------------------------------------------------------------------------
*** 5f74bab282cef8d18399c3ff, pen110 ***

------------------------------------------------------------------------------------------------
*** 5f74b82f82cef8d18399bfea, pen108 ***

------------------------------------------------------------------------------------------------
*** 5f6ac53982ce38aaa244973

In [10]:
#Check Array of Titles from the  catalog numbers above
arrSBTitle

['pen010',
 'pen007',
 'nmn926',
 'pen107',
 'pen116',
 'pen120',
 'pen117',
 'wup403',
 'wup413',
 'wup508',
 'pen102',
 'pen113',
 'wup411',
 'wup509',
 'wup114',
 'wup409',
 'wup516',
 'wup203',
 'wup410',
 'wup512',
 'pen015',
 'nmn803',
 'nmn902',
 'eup619',
 'nmn909',
 'nmn919',
 'eup613',
 'eup612',
 'eup523',
 'eup521',
 'eup608',
 'eup520',
 'pen118',
 'pen006',
 'pen119',
 'pen014',
 'nmn904',
 'wup118',
 'eup616',
 'eup627',
 'wup417',
 'wup517',
 'wup503',
 'eup626',
 'eup625',
 'eup615',
 'wup405',
 'wup5115',
 'wup104',
 'wup401',
 'nmn705',
 'wup219',
 'pen109',
 'pen012',
 'nmn708',
 'nmn903',
 'nmn905',
 'nmn901',
 'eup628',
 'wup211',
 'wup303',
 'wup308',
 'wup212',
 'wup515',
 'wup218',
 'eup518',
 'nmn911',
 'eup519',
 'nmn906',
 'nmn805',
 'eup629',
 'pen122',
 'pen103',
 'pen106',
 'pen105',
 'pen011',
 'wup206',
 'nmn913',
 'wup102',
 'pen121',
 'wup217',
 'nmn910',
 'wup208',
 'nmn923',
 'wup116',
 'wup215',
 'nmn908',
 'wup505',
 'wup406',
 'nmn801',
 'eup621'

In [11]:
#Produce directory listing of station (SB Object Children)
#Either set up the root directory with station subdirectories only or delete non-station directories from the list array
#mtDataDirList = os.walk(mtDataPath)
#mtDataDirList = [entry.path for entry in os.scandir(mtDataPath) if entry.is_dir()]
mtDataDirList = next(os.walk(mtDataPath))[1]
mtDataDirList

['eup518',
 'eup519',
 'eup520',
 'eup521',
 'eup523',
 'eup601',
 'eup602',
 'eup603',
 'eup604',
 'eup605',
 'eup607',
 'eup608',
 'eup609',
 'eup610',
 'eup612',
 'eup613',
 'eup614',
 'eup615',
 'eup616',
 'eup617',
 'eup618',
 'eup619',
 'eup620',
 'eup621',
 'eup622',
 'eup623',
 'eup624',
 'eup625',
 'eup626',
 'eup627',
 'eup628',
 'eup629',
 'eup630',
 'eup631',
 'nmn701',
 'nmn702',
 'nmn703',
 'nmn704',
 'nmn705',
 'nmn706',
 'nmn707',
 'nmn708',
 'nmn801',
 'nmn802',
 'nmn803',
 'nmn804',
 'nmn805',
 'nmn806',
 'nmn807',
 'nmn808',
 'nmn809',
 'nmn901',
 'nmn902',
 'nmn903',
 'nmn904',
 'nmn905',
 'nmn906',
 'nmn907',
 'nmn908',
 'nmn909',
 'nmn910',
 'nmn911',
 'nmn913',
 'nmn914',
 'nmn915',
 'nmn916',
 'nmn917',
 'nmn918',
 'nmn919',
 'nmn920',
 'nmn922',
 'nmn923',
 'nmn924',
 'nmn925',
 'nmn926',
 'pen006',
 'pen007',
 'pen008',
 'pen009',
 'pen010',
 'pen011',
 'pen012',
 'pen014',
 'pen015',
 'pen101',
 'pen102',
 'pen103',
 'pen104',
 'pen105',
 'pen106',
 'pen107',

## Now Upload the Files
##############################################################################
---------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# !!!!UPLOAD ALL FILES FOR THIS STATION TO SCIENCE BASE!!!!
# Previously uploaded files with the same name are automatically re-written
#this could take several minutes depoebding on the number of files
#the files size and the upload speed of the user's computer.
for j in range (len(mtDataDirList)):
 for k in range (len(arrSBTitle)):
    if mtDataDirList[j] == arrSBTitle[k]:
     mtStationPath = mtDataPath + '\\' + mtDataDirList[j]
     allFileList = glob.glob(os.path.join(mtStationPath, '*.*'),  recursive=True)
     item_json = sb.get_item(children[k])
     print(item_json['title'])
     for i in range(len(allFileList)):
      strExtension = allFileList[i].split('.')[1] 
      print("extension is: " + strExtension)
      if strExtension == 'mth5': #mth5 files too large to upload with API cahnge != to == to batch upload ONLY mth5 files
       item_json = sb.get_item(children[k])
       print ('Loading File: ' + allFileList[i])
       sb.replace_file(allFileList[i],item_json)
       time.sleep(3) # Sleep for 3 seconds
       print ('File: ' + allFileList[i] + '\n!!!!!! File Uploaded !!!!!!' )

print ('All Files Uploaded to ScienceBase')

station eup518
extension is: edi
extension is: mth5
Loading File: C:\CurrentWork\DataReleases\EU-Mike\child_xmls\eup518\eup518.mth5
File is too large.  Large files must be replaced with the UI.
